In [2]:
%load_ext autoreload
%autoreload 2


In [3]:
import sys
sys.path.append('/app')

In [4]:
from psql_helpers import create_share_link,accept_share_link

In [5]:
from psql_models import User, ShareLink, async_session,AccessLevel

In [6]:
user_id = 'ef7c085b-fdb5-4c94-b7b6-a61a3d04c210'
meeting_id = '2a40c363-c52b-4f89-8719-e0be87591485'

In [7]:
async with async_session() as session:
    link = await create_share_link(session, owner_id=user_id, meeting_ids=[meeting_id], expiration_hours=24, access_level=AccessLevel.TRANSCRIPT,
                      target_email='dmitrygrankin0@gmail.com')

    

In [8]:
from psql_models import ShareLink
from psql_helpers import read_table_async

In [9]:
await read_table_async(ShareLink)

,token,owner_id,target_email,access_level,created_at,expires_at,is_used,accepted_by,accepted_at,shared_meetings
0,RNpxT4m21qAvPM-UBglq0mQxqDY34CanEmUPIF9ys0M,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,dmitrygrankin0@gmail.com,transcript,2024-12-04 17:29:49.586646+00:00,2024-12-05 17:29:49.503948+00:00,True,f05bd139-3fa1-4563-a900-2516dffb67b3,2024-12-04 17:37:28.882825+00:00,[2a40c363-c52b-4f89-8719-e0be87591485]
1,AMnul043EtF2OMoLJ7cr1i43u-G1hKzz3xDhoWbIzZg,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,dmitrygrankin0@gmail.com,transcript,2024-12-04 18:14:16.749680+00:00,2024-12-05 18:14:16.668279+00:00,False,None,NaT,[2a40c363-c52b-4f89-8719-e0be87591485]


In [10]:
target_user_id_token = '918c9f3899d54f3bbcf445cd053c2304'
target_user_id = 'f05bd139-3fa1-4563-a900-2516dffb67b3'

In [11]:
async with async_session() as session:
    result = await accept_share_link(session=session, token=link, accepting_user_id=target_user_id, accepting_email='dmitrygrankin0@gmail.com')
    print(result)

IntegrityError: (sqlalchemy.dialects.postgresql.asyncpg.IntegrityError) <class 'asyncpg.exceptions.UniqueViolationError'>: duplicate key value violates unique constraint "uq_user_meeting"
DETAIL:  Key (meeting_id, user_id)=(2a40c363-c52b-4f89-8719-e0be87591485, f05bd139-3fa1-4563-a900-2516dffb67b3) already exists.
[SQL: INSERT INTO user_meetings (meeting_id, user_id, access_level, created_at, created_by, is_owner) VALUES ($1::UUID, $2::UUID, $3::VARCHAR, $4::TIMESTAMP WITH TIME ZONE, $5::UUID, $6::BOOLEAN) RETURNING user_meetings.id]
[parameters: (UUID('2a40c363-c52b-4f89-8719-e0be87591485'), 'f05bd139-3fa1-4563-a900-2516dffb67b3', 'transcript', datetime.datetime(2024, 12, 4, 18, 14, 21, 443496, tzinfo=datetime.timezone.utc), UUID('ef7c085b-fdb5-4c94-b7b6-a61a3d04c210'), False)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [12]:
accept_share_link()

TypeError: accept_share_link() missing 3 required positional arguments: 'session', 'token', and 'accepting_user_id'

In [13]:
link

'AMnul043EtF2OMoLJ7cr1i43u-G1hKzz3xDhoWbIzZg'

In [14]:
from api_client import APIClient,MeetingOwnership


In [15]:

client = APIClient(email='dmitrygrankin0@gmail.com')
await client.get_meetings(ownership=MeetingOwnership.ALL)

ConnectionError: HTTPConnectionPool(host='localhost', port=8765): Max retries exceeded with url: /meetings/all?offset=0&limit=50&include_summary=False&ownership=all (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f620d1abb90>: Failed to establish a new connection: [Errno 111] Connection refused'))